<a href="https://colab.research.google.com/github/serinss/seSAC_ML_DL_Class/blob/main/05_2_%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%run my_init.py

Mounted at /content/drive


In [2]:
%run my_init.py

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#교차 검증
딥러닝 에서는 교차검증 하지 않음  
비정형 데이터 대상이므로 크기가 월등히 커서 시간이 많이 걸림  

교차 검증 장점 : 

1. 모든 데이터 셋을 평가에 활용할 수 있다.

  - 평가에 사용되는 데이터 편중을 막을 수 있다. 

(특정 평가 데이터 셋에 overfit 되는 것을 방지할 수 있다.)

  - 평가 결과에 따라 좀 더 일반화된 모델을 만들 수 있다. 

2. 모든 데이터 셋을 훈련에 활용할 수 있다.

  - 정확도를 향상시킬 수 있다.

  - 데이터 부족으로 인한 underfitting을 방지할 수 있다.

​

교차 검증 단점 : Iteration 횟수가 많기 때문에 모델 훈련/평가 시간이 오래 걸린다.

In [3]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [5]:
data = wine[['alcohol','sugar','pH']].to_numpy()
data.shape

(6497, 3)

In [4]:

target = wine['class'].to_numpy()
target.shape

(6497,)

In [6]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [7]:
train_input.shape, train_target.shape

((5197, 3), (5197,))

In [8]:
test_input.shape, test_target.shape

((1300, 3), (1300,))

In [9]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)

In [11]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target, cv=3) # estimator, X, y=None
print(scores)

{'fit_time': array([0.00834584, 0.00635171, 0.00604248]), 'score_time': array([0.00094795, 0.00081754, 0.00079703]), 'test_score': array([0.84535488, 0.87182448, 0.84757506])}


In [12]:
np.mean(scores['test_score']) # 정확히는 validation score

0.8549181380146386

In [13]:
for k in scores.keys():
  print(k)

fit_time
score_time
test_score


In [15]:
scores = cross_validate(dt, train_input, train_target, cv=3,  return_train_score=True)
print(scores)

{'fit_time': array([0.00905108, 0.00614643, 0.00606513]), 'score_time': array([0.00099349, 0.00082588, 0.00081134]), 'test_score': array([0.84535488, 0.87182448, 0.84757506]), 'train_score': array([0.99711316, 0.9979798 , 0.9979798 ])}


In [17]:
np.mean(scores['test_score'])

0.8549181380146386

In [16]:
np.mean(scores['train_score']) #overfit 상태

0.9976909199772942

#K Fold 객체 사용

In [24]:
from sklearn.model_selection import KFold, StratifiedKFold

#n_splits : 몇 폴드로 교차검증을 할 지 선택 -> 무조건 첫번째 파라미터
kfold = KFold(3, shuffle=True, random_state=42) # 랜덤방식 -> 데이터가 많으면 편향이 발생하기 쉬움 -> 층화 임의 추출(StratifiedKFold)
skfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) 

In [21]:
scores = cross_validate(dt, train_input, train_target, cv=kfold, return_train_score=True)
print(scores)

{'fit_time': array([0.00872421, 0.00608993, 0.00653934]), 'score_time': array([0.00094628, 0.000772  , 0.00123501]), 'test_score': array([0.84997115, 0.84237875, 0.84930716]), 'train_score': array([0.99711316, 0.9965368 , 0.9994228 ])}


In [25]:
scores = cross_validate(dt, train_input, train_target, cv=skfold, return_train_score=True)
print(scores)

{'fit_time': array([0.00855637, 0.00606561, 0.00612378]), 'score_time': array([0.0012908 , 0.00079799, 0.00083947]), 'test_score': array([0.85920369, 0.85450346, 0.84237875]), 'train_score': array([0.99682448, 0.998557  , 0.9979798 ])}


In [39]:
for i,v in enumerate(np.arange(0.0003, 0.0008, 0.0001)):
  dt = DecisionTreeClassifier(min_impurity_decrease=v, random_state=42)
  scores = cross_validate(dt, train_input, train_target, cv=skfold, return_train_score=True )
  print(f'[{i}]')
  print(f"train score: {np.mean(scores['train_score'])}")
  print(f"test score: {np.mean(scores['test_score'])}", end = '\n\n')

# 점수들이 올라가다가 다시 내려감
  # min_impurity_decrease 값 0.0001 ~ 0.0010 이러한 값들은 경험적인 값들 -> 경험이 없으면 알 수 없음

[0]
train score: 0.9295742257058653
test score: 0.8637674370670857

[1]
train score: 0.9119681362175589
test score: 0.863958782267153

[2]
train score: 0.9000382412045229
test score: 0.8654987613091344

[3]
train score: 0.8947468748854431
test score: 0.8668459514554007

[4]
train score: 0.8869535562376208
test score: 0.8631911803966564



#하이퍼 파라미터 튜닝

In [46]:
from sklearn.model_selection import GridSearchCV

dt = DecisionTreeClassifier(random_state=42)
params = {
    "max_depth": range(3, 6),
    "min_samples_split": [2,5,8],
    "min_impurity_decrease": np.arange(0.0001, 0.0006, 0.0001)
}

In [44]:
3 * 3 * 5 * 5
# depth * split * impurity * fold 모두 곱하면 갯수가 나옴

225

In [47]:
gs = GridSearchCV(dt, params, cv=5, n_jobs=-1)
# estimator, param_grid * 별표 앞의 파라미터는 positional argument -> 순서대로 적어야함
# * 뒤에 오는 파라미터는 keyword argument -> 이름만 제대로 적으면 알아서 들어감

gs.fit(train_input, train_target)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(3, 6),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005]),
                         'min_samples_split': [2, 5, 8]})

In [55]:
gs.best_estimator_ #best 값은 저장을 해 두었으니 우리는 꺼내서 참조하면 됨

DecisionTreeClassifier(max_depth=5,
                       min_impurity_decrease=0.00030000000000000003,
                       random_state=42)

In [56]:
gs.best_params_

{'max_depth': 5,
 'min_impurity_decrease': 0.00030000000000000003,
 'min_samples_split': 2}

In [57]:
gs.best_score_ # test score

0.8585735174354039

In [59]:
gs.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_impurity_decrease', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [60]:
gs.cv_results_['mean_test_score']

array([0.84125583, 0.84125583, 0.84125583, 0.84125583, 0.84125583,
       0.84125583, 0.84125583, 0.84125583, 0.84125583, 0.84125583,
       0.84125583, 0.84125583, 0.84125583, 0.84125583, 0.84125583,
       0.85337806, 0.85337806, 0.85337806, 0.85337806, 0.85337806,
       0.85337806, 0.85337806, 0.85337806, 0.85337806, 0.85337806,
       0.85337806, 0.85337806, 0.85318557, 0.85318557, 0.85318557,
       0.85780355, 0.85780355, 0.85799604, 0.85799604, 0.85799604,
       0.85799604, 0.85857352, 0.85857352, 0.85857352, 0.85857352,
       0.85857352, 0.85857352, 0.85838102, 0.85838102, 0.85838102])

In [61]:
np.max(gs.cv_results_['mean_test_score']) # gs.best_score_ 와 동일

0.8585735174354039

In [62]:
best_model = gs.best_estimator_
best_model.get_params() #해당 베스트 모델의 hyper parameter 정보 꺼내오기

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.00030000000000000003,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

In [63]:
best_model.score(train_input, train_target) 

0.8668462574562248

In [64]:
best_model.score(test_input, test_target) # 아주 조금 낮음 - Decision Tree는 한계가 있음을 알 수 있음 -> 점수 잘 안오름
# overfit는 방지 했지만 점수 높지 않음

0.8623076923076923

In [41]:
# 책 예제

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # 5x5 = 25개 모델 fit()
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [42]:
print(gs.best_params_), print(gs.cv_results_['mean_test_score'])

{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


(None, None)

#확률 분포 선택 / 랜덤 서치
그리드 서치가 아니라 분포를 이용한 서치를 실행

In [65]:
from scipy.stats import uniform, randint

rgen = randint(0, 10) # 균등분포를 따르는 정수값인 난수를 생성하는 함수
rgen.rvs(10) # 10개 뽑기 - 복원추출

array([0, 2, 9, 6, 9, 5, 6, 5, 4, 9])

In [66]:
np.unique(rgen.rvs(1000), return_counts=True)
# 1000개 정도 뽑았을 때, 균등하게 나옴

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 93, 100, 109,  94, 102,  98, 107,  91, 108,  98]))

In [67]:
ugen = uniform(0,1) # 균등 분포를 따르는 실수값 생성
ugen.rvs(10)

# 난수 발생기와 유사함

array([0.51856791, 0.05746335, 0.38529838, 0.19495707, 0.96391295,
       0.3552058 , 0.32985386, 0.46612002, 0.99733113, 0.41380601])